# HW 5 - Neural Machin Translator

### Install dependencies


In [2]:
%%bash
pip install fastBPE regex requests sacremoses subword_net
git clone https://github.com/pytorch/fairseq.git
cd fairseq && git checkout 9a1c497
pip install --editable ./
pip install bitarray
pip install configargparse
pip install torchtext==0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=452871432da02b9af8b03fa471507ca981bc1417a064bed5f3cfce8be6a2f434
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninsta

ERROR: Could not find a version that satisfies the requirement subword_net (from versions: none)
ERROR: No matching distribution found for subword_net
Cloning into 'fairseq'...
Note: checking out '9a1c497'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 9a1c4970 Make Hydra logging work with DDP (#1568)


In [3]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d54ec41e2c756884d7c60469f1e05b2880692baad1f8e26916c1e5f650602c61
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [4]:
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.3 MB/s 


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 


In [6]:
!pip install parsivar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 375 kB/s 
     |████████████████████████████████| 1.5 MB 36.3 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=ce39d0987190058e6638946dc15b2f1f39c84112c721a61e41c17d35e544376e
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449921 sha256=2fe60b02b6aacaeb73162bd1e13b4c2c11bf46b6ba858ceba4de305c748f9429
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


### Mounting Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import libraries - Files

In [8]:
from __future__ import unicode_literals
from parsivar import *
import pandas as pd
import numpy as np
import math
import torch
import re
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
!cp  "/content/drive/MyDrive/Sheyda/Data - NMT/train.en" .
!cp "/content/drive/MyDrive/Sheyda/Data - NMT/test.en" .
!cp "/content/drive/MyDrive/Sheyda/Data - NMT/valid.en" .
!cp "/content/drive/MyDrive/Sheyda/Data - NMT/train.fa" .
!cp "/content/drive/MyDrive/Sheyda/Data - NMT/test.fa" .
!cp "/content/drive/MyDrive/Sheyda/Data - NMT/valid.fa" .

### Preprocessing - Cleaning Data

In [13]:
from parsivar import Normalizer
from parsivar import SpellCheck
parsivar_normalizer = Normalizer(pinglish_conversion_needed=True)
parsivar_tokenizer = Tokenizer()

def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

def farsi_preprocess():
  farsi_files_to_be_preprocessed = ["train.fa", "valid.fa", "test.fa"]
  for files in farsi_files_to_be_preprocessed:
    lines = []
    with open(files) as file:
        for line in file:
          line = ''.join([' ' + char if i > 0 and no_space(char, line[i - 1]) else char for i, char in enumerate(line)])
          line = re.sub('[)(}{-]', ' ', line)
          lines.append(' '.join(parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(line.rstrip())))) #word_tokenize in parsivar
        with open(f"preprocessed_{files}", 'w') as f:
          f.write('\n'.join(lines))

def english_preprocess():
  english_files_to_be_preprocessed = ["train.en", "valid.en", "test.en"]
  for files in english_files_to_be_preprocessed:
    lines = []
    with open(files) as file:
        for line in file:
          line = ''.join([' ' + char if i > 0 and no_space(char, line[i - 1]) else char for i, char in enumerate(line)])
          line = re.sub('[)(}{-]', ' ', line)
          lines.append(' '.join(nltk.word_tokenize(' '.join([word.lower() for word in line.split(' ')]))))
        with open(f"preprocessed_{files}", 'w') as f:
          f.write('\n'.join(lines))

farsi_preprocess()
english_preprocess()

### Fairseq Preprocessing - Using BPE

In [14]:
!fairseq-preprocess --workers 20 --joined-dictionary \
--source-lang en --target-lang fa --bpe byte_bpe --optimizer nag --trainpref /content/preprocessed_train --validpref /content/preprocessed_valid --testpref /content/preprocessed_test --destdir data-bin/custom.tokenized.en-fa

2022-06-09 10:48:09 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe='byte_bpe', checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/custom.tokenized.en-fa', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='en', srcdict=None, target_lang='fa', task='translation', tensorboard_logdir=None, testpref='/content/preprocessed_test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt

Remove temp files

In [15]:
#!rm -rf checkpoints

Make folder for saving checkpoints

In [16]:
!mkdir -p checkpoints/models

### Fairseq Train - Conv Architecture

In [17]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train data-bin/custom.tokenized.en-fa \
   --optimizer nag --lr 0.3 --clip-norm 0.25 --dropout 0.2 --max-tokens 6000 --fp16 --max-epoch 25\
   --arch mbart_base --save-dir checkpoints/models 

2022-06-09 10:48:27 | INFO | fairseq_cli.train | Namespace(activation_fn='gelu', adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='mbart_base', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.25, cpu=False, criterion='cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/custom.tokenized.en-fa', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=12, decoder_embed_dim=768, decoder_embed_path=None, decoder_ffn_embed_dim=3072, decoder_input_dim=768, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=True, decoder_normalize_before=False, decoder_output_dim=768, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distribut

### Fairseq Generation

In [18]:
!fairseq-generate /content/data-bin/custom.tokenized.en-fa \
    --path /content/checkpoints/models/checkpoint_best.pt \
    --batch-size 512 --beam 4 \
    --skip-invalid-size-inputs-valid-test

2022-06-09 11:38:57 | INFO | fairseq_cli.generate | Namespace(all_gather_list_size=16384, batch_size=128, batch_size_valid=128, beam=3, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/content/data-bin/custom.tokenized.en-fa', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_

### Interactive Generation

In [19]:
!curl https://dl.fbaipublicfiles.com/fairseq/models/wmt14.v2.en-fr.fconv-py.tar.bz2 | tar xvjf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0wmt14.en-fr.fconv-py/
wmt14.en-fr.fconv-py/model.pt
 99 1909M   99 1903M    0     0  10.0M      0  0:03:10  0:03:09  0:00:01 10.2Mwmt14.en-fr.fconv-py/dict.en.txt
wmt14.en-fr.fconv-py/dict.fr.txt
100 1909M  100 1909M    0     0  10.0M      0  0:03:10  0:03:10 --:--:-- 10.3M
wmt14.en-fr.fconv-py/bpecodes
wmt14.en-fr.fconv-py/README.md


In [22]:
!fairseq-interactive \
    --path /content/checkpoints/models/checkpoint_best.pt /content/data-bin/custom.tokenized.en-fa \
    --beam 4 --source-lang en --target-lang fa \
    --tokenizer moses \

2022-06-09 12:40:27 | INFO | fairseq_cli.interactive | Namespace(all_gather_list_size=16384, batch_size=1, batch_size_valid=None, beam=4, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, buffer_size=1, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='/content/data-bin/custom.tokenized.en-fa', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, 